In [ ]:
%cd /content/drive/My Drive/Colab Notebooks/data
import json
import pandas as pd
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
import spacy 
import re
from sklearn.model_selection import train_test_split as splitter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.callbacks import PerplexityMetric
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Embedding, LSTM ,Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Activation, RepeatVector,Flatten, TimeDistributed, Input,Bidirectional,LocallyConnected1D,Conv1D,GlobalAveragePooling1D,GlobalMaxPooling1D,Concatenate,BatchNormalization
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras import layers
from sklearn.feature_extraction.text import CountVectorizer

/content/drive/My Drive/Colab Notebooks/data
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
#loading lyrics
lyrics = pd.read_csv('data.csv')
lyrics_data=list(lyrics['lyrics'].str.lower())
artist_data=list(lyrics['artist'])

In [ ]:
#pre processing
def preprocessing(data):
  processed_data=[]

  #removing everything except numbers and words
  for i in data:
    x = re.sub(r'[^a-z]+',' ',i)
    processed_data.append(x)

  #tokenizing sentences
  tokenized = [list(word_tokenize(i))[:100] for i in processed_data]

  
  '''
  #removing stopword
  stop_words= list(set(stopwords.words('english')))
  middle=[]
  for ind,i in enumerate(tokenized):
    for index,j in enumerate(i):
      if j not in stop_words:
        middle.append(j)
    tokenized[ind]=middle
    middle=[]
    '''
  '''
  #removing words with len < 2
  middle=[]
  for ind,i in enumerate(tokenized):
    for index,j in enumerate(i):
      if len(j) > 2:
        middle.append(j)
    tokenized[ind]=middle
    middle=[] 
  
  #lemmatizing
  lemmatizer = WordNetLemmatizer() 
  middle=[]
  for ind,i in enumerate(tokenized):
    for index,j in enumerate(i):
      middle.append(lemmatizer.lemmatize(j))
  tokenized[ind]=middle
  middle=[]
  '''

  


  
  
  return tokenized

  
processed_data=preprocessing(lyrics_data)



In [ ]:
#splitting data in test,valid,train 20 10 70
def split(data,labels):
  train_data,valid_data,train_labels,valid_labels=splitter(data,labels,train_size=0.9,test_size=0.1,shuffle=True)
  train_data,test_data,train_labels,test_labels=splitter(train_data,train_labels,train_size=0.8,test_size=0.2,shuffle=True)

  return train_data,train_labels,test_data,test_labels,valid_data,valid_labels

train_data,train_labels,test_data,test_labels,valid_data,valid_labels=split(processed_data,artist_data)


In [ ]:
#one hot encoding labels
#Eminem G-eazy Kendrick MGK
def one_hot(labels):
  label_encoder = LabelEncoder()
  integer_encoded = label_encoder.fit_transform(labels)
  onehot_encoder = OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  onehot_encoded= onehot_encoder.fit_transform(integer_encoded)
  return np.array(onehot_encoded)

encoded_train_labels = one_hot(train_labels)
encoded_test_labels  = one_hot(test_labels)
encoded_valid_labels = one_hot(valid_labels)


In [ ]:
#callbacks
class EpochLogger(CallbackAny2Vec):
  def __init__(self):
    self.epoch = 0
  def on_epoch_begin(self, model):
    print("Epoch #{} start".format(self.epoch))
  def on_epoch_end(self, model):
    print("Epoch #{} end".format(self.epoch))
    self.epoch += 1

#word2vec
epoch_logger = EpochLogger()
perplexity_logger = PerplexityMetric(corpus=train_data, logger='shell')
model = Word2Vec(train_data, size=200, window=3,iter=1000)
model.save("word2vec.model")


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
#loading pretrained embeddings
path = '/content/drive/My Drive/Colab Notebooks/data/glove.6B.100d.txt'
with open(path,'r')as f:
  lines=f.readlines()
  f.close()
emb={}
for i in lines:
  line=i.split(" ")
  word=line[0]
  emb[word]=[float(j) for j in line[1:]]
#adding 0
emb['UNK']=np.zeros(100,dtype='float')

# __CNN__ implementation

In [ ]:
#get vocab size
def getvocab(data):
  raw_count = 0
  vocab = []
  for i in data:
    for j in i:
      raw_count +=1
      if j not in vocab:
        vocab.append(j)
  return raw_count,vocab

raw_count,vocab = getvocab(train_data)


In [ ]:
raw_count


28696

In [ ]:
#pooling functions
#Fast text averaging, pass a list of word embeddings and embedding size to fasttextAveraging function
def l2_norm(x):
   return np.sqrt(np.sum(x**2))

def div_norm(x):
   norm_value = l2_norm(x)
   if norm_value > 0:
       return x * ( 1.0 / norm_value)
   else:
       return x
def fasttextAveraging(embedding_list,embedding_size):
    norm=np.zeros(embedding_size)
    for emb in embedding_list:
        norm=norm+div_norm(emb) 
    return norm/len(embedding_list)

def averagePooling(embedding_list,embedding_size):
  return np.mean(embedding_list,axis=0)

In [ ]:
#making embeddings
def embed(model,data,size):
  emb_size = size
  middle=[]
  matrix =np.zeros((len(data),emb_size))
  for index,i in enumerate(data):
    for j in i:
      try:
        middle.append(model.wv[j])
      except:
        continue
    middle = averagePooling(middle,emb_size)
    matrix[index]=middle
    middle=[]
  return matrix

train_matrix = embed(emb,train_data,100)
test_matrix  = embed(emb,test_data,100)
vaid_matrix  = embed(emb,valid_data,100)


/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
#modelling
#keras sequential api
embedding_size=100
"""model_word2vec = keras.Sequential([
    keras.layers.Flatten(input_shape=(embedding_size,)),#donot change(input layer)
    keras.layers.Dense(100, activation='relu'),#(hidden layer)
    keras.layers.Dense(50, activation='relu'),#(hidden layer)
    keras.layers.Dense(4)#donot change
])
adam=keras.optimizers.Adam(lr=0.00001)
model_word2vec.compile(optimizer=adam,
              loss=["categorical_crossentropy"],
              metrics=['accuracy'])

model_word2vec.summary()"""

model1 = Sequential()

model1.add(layers.Embedding(vocab_size+1, embedding_size, input_length=100))
model1.add(layers.Conv1D(128, 5, activation='relu'))
model1.add(layers.GlobalMaxPooling1D())
model1.add(layers.Dense(10, activation='relu'))
model1.add(layers.Dense(4, activation='sigmoid'))
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model1.summary()


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 100, 100)          382500    
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 96, 128)           64128     
_________________________________________________________________
global_max_pooling1d_6 (Glob (None, 128)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                1290      
_________________________________________________________________
dense_14 (Dense)             (None, 4)                 44        
Total params: 447,962
Trainable params: 447,962
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#callbacks
filepath = "setting_" + "model1" + ".hdf5"
logfilepath = "setting_"+"model1" + ".csv"
reduce_lr_rate=0.2
logCallback = CSVLogger(logfilepath, separator=',', append=False)
earlyStopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_weights_only=True, verbose=1,
                             save_best_only=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_rate, patience=10,
                              cooldown=0, min_lr=0.0000000001, verbose=0)

callbacks_list = [logCallback, reduce_lr, checkpoint]


In [ ]:
#training
model1.fit(train_matrix,encoded_train_labels, epochs=100, batch_size=25,verbose=1,shuffle=True,validation_data=(vaid_matrix,encoded_valid_labels),callbacks = callbacks_list)

Epoch 1/100


InvalidArgumentError: ignored

# __RNN__ implementation

In [ ]:
#getting vocab 
def vocab(data):
  tokenizer=Tokenizer(oov_token="UNK")
  tokenizer.fit_on_texts(data)
  vocab = list(tokenizer.word_index.keys())
  vocab_size = len(vocab)
  return tokenizer,vocab,vocab_size
tokenizer,vocab,vocab_size = vocab(train_data)

In [ ]:
len(vocab)

3885

In [ ]:
#converting to sequences and adding post padding
def text2seq(tokenizer,data):

  seq=tokenizer.texts_to_sequences(data)
  #padding
  seq=pad_sequences(seq,padding="post",maxlen=100)
  return seq

train_seq = text2seq(tokenizer,train_data)
valid_seq = text2seq(tokenizer,valid_data)
test_seq  = text2seq(tokenizer,test_data)

In [ ]:
#creating embeddings 
def embed(model,vocab,Emb_dim):
  emb_matrix = np.zeros((len(vocab)+1,Emb_dim))
  for index,i in enumerate(vocab):
      try:
          emb_matrix[index]=model[i]
      except:
        continue
  return emb_matrix

embeddings =embed(emb,vocab,100)

In [ ]:
filepath = "setting_" + "model1" + ".hdf5"
logfilepath = "setting_"+"model1" + ".csv"
reduce_lr_rate=0.2
logCallback = CSVLogger(logfilepath, separator=',', append=False)
earlyStopping = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=10, verbose=0, mode='auto')
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', save_weights_only=True, verbose=1,
                             save_best_only=True, mode='auto')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=reduce_lr_rate, patience=10,
                              cooldown=0, min_lr=0.0000000001, verbose=0)

callbacks_list = [logCallback, reduce_lr, checkpoint,earlyStopping]

In [ ]:
#initializing models
Emb_dim = 100
input_layer=Input(shape=(100,))
emb=Embedding(vocab_size+1,Emb_dim,weights=[embeddings],input_length=100,trainable=True)
emb_out = emb(input_layer)
lstm=LSTM(25,return_state=True,return_sequences=True)
lstm_out,_,_ = lstm(emb_out)
drop1=Dropout(0.5)(lstm_out)
lstm2 = LSTM(25,return_state=False,return_sequences=False,kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01))
lstm2_out=lstm2(drop1)
drop2 = Dropout(0.5)(lstm2_out)
output=Dense(4,activation='sigmoid')(drop2)

model1 = Model(inputs=[input_layer], outputs=[output])
model1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])
model1.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 100)          382500    
_________________________________________________________________
lstm (LSTM)                  [(None, 100, 25), (None,  12600     
_________________________________________________________________
dropout (Dropout)            (None, 100, 25)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 25)                5100      
_________________________________________________________________
dropout_1 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense (Dense)                (None, 4)                 104   

In [ ]:

model1.fit(train_seq,encoded_train_labels, epochs=100,validation_data=(valid_seq,encoded_valid_labels),verbose=1,shuffle=True,callbacks=callbacks_list)

Epoch 1/100
9/9 [==============================] - 1s 65ms/step - loss: 0.9694 - categorical_accuracy: 0.8333 - val_loss: 2.1911 - val_categorical_accuracy: 0.2500 - lr: 0.0010
Epoch 2/100
9/9 [==============================] - 1s 64ms/step - loss: 0.9134 - categorical_accuracy: 0.7986 - val_loss: 2.1244 - val_categorical_accuracy: 0.3750 - lr: 0.0010
Epoch 3/100
9/9 [==============================] - 1s 67ms/step - loss: 0.8766 - categorical_accuracy: 0.8368 - val_loss: 2.0700 - val_categorical_accuracy: 0.3500 - lr: 0.0010
Epoch 4/100
9/9 [==============================] - 1s 64ms/step - loss: 0.8433 - categorical_accuracy: 0.8403 - val_loss: 2.1026 - val_categorical_accuracy: 0.2750 - lr: 0.0010
Epoch 5/100
9/9 [==============================] - 1s 63ms/step - loss: 0.8273 - categorical_accuracy: 0.7882 - val_loss: 2.1130 - val_categorical_accuracy: 0.3750 - lr: 0.0010
Epoch 6/100
9/9 [==============================] - 1s 64ms/step - loss: 0.7815 - categorical_accuracy: 0.8472 - val

KeyboardInterrupt: ignored